In [ ]:
!apt-get update && apt-get install swig cmake
!pip install box2d-py
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install sb3-contrib

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import DQN
from sb3_contrib.qrdqn import QRDQN
from stable_baselines3.common.evaluation import evaluate_policy
import os
from stable_baselines3.common.monitor import Monitor
import matplotlib.pyplot as plt
from stable_baselines3.common.results_plotter import plot_results, ts2xy
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from IPython import display as ipythondisplay
import base64
from pathlib import Path

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


Let's evaluate the un-trained agent, this should be a random agent.

In [ ]:
model = QRDQN("CnnPolicy", ENVIROMENT,verbose=1 , exploration_final_eps=0.1, target_update_interval=250,)
# Separate env for evaluation
eval_env = gym.make(ENVIROMENT)

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True,)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

Train the agent and save it

In [ ]:
#  "MlpPolicy"  "CnnPolicy"

# enviroment  = 'MsPacman-v0'   "LunarLander-v2"   "ALE/MsPacman-v5"   "CartPole-v0"

ENVIROMENT = 'CartPole-v0'
POLICY = "MlpPolicy"

# Train the agent
log_dir = "temp1"
os.makedirs(log_dir, exist_ok=True)

env = gym.make(ENVIROMENT)
env = Monitor(env, log_dir)

policy_kwargs = dict(n_quantiles=15)

model = QRDQN(POLICY, env, policy_kwargs=policy_kwargs, verbose=1 , batch_size = 32 )
model.learn(total_timesteps=int(40000), log_interval=4)

# Save the agent
model.save(f"QR_dqn_{ENVIROMENT}")
del model  # delete trained model to demonstrate loading
model = QRDQN.load(f"QR_dqn_{ENVIROMENT}")

Using cpu device
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.8     |
|    ep_rew_mean      | 22.8     |
|    exploration_rate | 0.55     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3920     |
|    time_elapsed     | 0        |
|    total_timesteps  | 91       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.1     |
|    ep_rew_mean      | 21.1     |
|    exploration_rate | 0.163    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4782     |
|    time_elapsed     | 0        |
|    total_timesteps  | 169      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.3     |
|    ep_rew_mean      | 22.3     |
|    exploration_rate | 0.01     |
| t

Load the trained agent

In [ ]:
# Evaluate the trained agent
eval_env = gym.make(ENVIROMENT)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=9.40 +/- 0.7999999999999999


In [ ]:
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')


def plot_results(log_folder, title='Learning Curve'):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), 'timesteps')
    y = moving_average(y, window=50)
    # Truncate x
    x = x[len(x) - len(y):]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title + " Smoothed")
    plt.show()

In [ ]:
def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(ENVIROMENT, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
record_video(ENVIROMENT, model, video_length=500, prefix="L-L")

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Saving video to /content/videos/L-L-step-0-to-step-500.mp4
Moviepy - Building video /content/videos/L-L-step-0-to-step-500.mp4.
Moviepy - Writing video /content/videos/L-L-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/L-L-step-0-to-step-500.mp4


In [ ]:
show_videos("videos", prefix="L-L")